In [0]:
%pip install azure-eventhub

In [0]:
dbutils.library.restartPython()

In [0]:
storage_account_name = "anilrtlpstg"
access_key = "muGpHzhXHViflMEuSjPpG498HGi9hW11vHFGp/1nyEMlQtsEh5NoOi0Xyr1alXh6o2FjD7ox4nM9+AStGPHsOw=="

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    access_key
)

In [0]:
dbutils.fs.mkdirs("abfss://streaming@anilrtlpstg.dfs.core.windows.net/test")

In [0]:
connectionString = "Endpoint=sb://anil-rtlp-ehns.servicebus.windows.net/;SharedAccessKeyName=telemetry-policy;SharedAccessKey=tmGw2UkOTGO7GFgbuN92jpqolDBg+eEla+AEhHe/tZw=;EntityPath=telemetry-hub"

ehConf = {
  'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString),
  'eventhubs.startingPosition': '{"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}'
}

In [0]:
df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load()

In [0]:
from pyspark.sql.functions import col

json_df = df.selectExpr("CAST(body AS STRING) as json")

In [0]:
from pyspark.sql.types import *

telemetry_schema = StructType([
    StructField("deviceId", StringType()),
    StructField("temperature", DoubleType()),
    StructField("humidity", DoubleType()),
    StructField("timestamp", StringType())
])

In [0]:
from pyspark.sql.functions import from_json

parsed_df = json_df.select(
    from_json(col("json"), telemetry_schema).alias("data")
).select("data.*")

In [0]:
query = parsed_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://streaming@anilrtlpstg.dfs.core.windows.net/checkpoint/final") \
    .start("abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events")

In [0]:
display(dbutils.fs.ls("abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events"))

In [0]:
spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events"
).display()

In [0]:
# ===============================
# CELL 14 — Read Bronze Delta data
# Bronze = raw streaming data from EventHub
# ===============================

bronze_df = spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events"
)

display(bronze_df)

In [0]:
# ===============================
# CELL 15 — Silver transformation
# Cleaning + datatype conversion
# ===============================

from pyspark.sql.functions import to_timestamp, col, to_date

silver_df = bronze_df \
    .filter(col("deviceId").isNotNull()) \
    .withColumn("event_time", to_timestamp("timestamp")) \
    .withColumn("event_date", to_date("event_time"))

display(silver_df)

In [0]:
# ===============================
# CELL 16 — Write Silver Delta
# ===============================

silver_df.write.format("delta") \
.mode("overwrite") \
.save("abfss://streaming@anilrtlpstg.dfs.core.windows.net/silver/events")

In [0]:
# ===============================
# CELL 17 — Validate Silver table
# ===============================

display(
spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/silver/events"
))

In [0]:
# ============================================
# GOLD LAYER — Streaming aggregation
# Business-level metrics per device
#
# Metrics:
#  - Average temperature per device
#  - Average humidity per device
#  - Total number of events per device
# ============================================

from pyspark.sql.functions import avg, count

gold_df = parsed_df.groupBy("deviceId").agg(
    avg("temperature").alias("avg_temperature"),
    avg("humidity").alias("avg_humidity"),
    count("*").alias("event_count")
)

In [0]:
# ============================================
# GOLD LAYER — Write aggregated results to Delta
#
# Streaming aggregation uses COMPLETE mode because
# groupBy results must be rewritten every trigger
#
# Checkpoint ensures fault tolerance and recovery
# ============================================

gold_query = gold_df.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", "abfss://streaming@anilrtlpstg.dfs.core.windows.net/checkpoint/gold") \
    .start("abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/gold")

In [0]:
# ============================================
# GOLD LAYER — Validate aggregated table
# Reading Delta table to confirm metrics
# ============================================

spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/gold"
).display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold_metrics
USING DELTA
AS SELECT *
FROM delta.`abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/gold`

In [0]:
%sql
SELECT * FROM gold_metrics

In [0]:
for s in spark.streams.active:
    s.stop()